In [2]:
import pandas as pd
import pickle
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Load the data from the pickled file
file_path = '/content/drive/My Drive/synthetic_dataset/goal_set.p'
with open(file_path, 'rb') as file:
    consultation_data = pickle.load(file)

test_data = consultation_data['test']

Mounted at /content/drive


In [3]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Extract explicit and implicit symptoms along with their disease tags
disease_tags = []
explicit_symptoms_list = []
implicit_symptoms_list = []

for item in test_data:
    disease_tags.append(item['disease_tag'])
    explicit_symptoms = list(item['goal']['explicit_inform_slots'].keys())
    implicit_symptoms = list(item['goal']['implicit_inform_slots'].keys())
    explicit_symptoms_list.append(", ".join(explicit_symptoms))
    implicit_symptoms_list.append(", ".join(implicit_symptoms))

# Create DataFrame
df = pd.DataFrame({
    'Disease Tag': disease_tags,
    'Explicit Symptoms': explicit_symptoms_list,
    'Implicit Symptoms': implicit_symptoms_list
})

# Define and train the models
X = df['Explicit Symptoms'] + " " + df['Implicit Symptoms']
y = df['Disease Tag']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize text data
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Define models
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42)
}

# Define NDCG calculation functions
def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gain = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)

def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best

# Train, predict and evaluate each model
results = {}

for model_name, model in models.items():
    # Train model
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate NDCG@10 and NDCG@5
    y_test_bin = MultiLabelBinarizer().fit_transform([[label] for label in y_test])
    y_score = model.predict_proba(X_test)
    ndcg_scores_10 = [ndcg_score(y_test_bin[i], y_score[i], k=10) for i in range(len(y_test))]
    ndcg_scores_5 = [ndcg_score(y_test_bin[i], y_score[i], k=5) for i in range(len(y_test))]
    mean_ndcg_10 = np.mean(ndcg_scores_10)
    mean_ndcg_5 = np.mean(ndcg_scores_5)

    # Store results
    results[model_name] = {
        'Accuracy': accuracy,
        'NDCG@10': mean_ndcg_10,
        'NDCG@5': mean_ndcg_5
    }

# Display results
for model_name, metrics in results.items():
    print(f"{model_name} - Accuracy: {metrics['Accuracy']:.2f}, NDCG@10: {metrics['NDCG@10']:.5f}, NDCG@5: {metrics['NDCG@5']:.5f}")


Random Forest - Accuracy: 0.75, NDCG@10: 0.86821, NDCG@5: 0.86349
Logistic Regression - Accuracy: 0.78, NDCG@10: 0.89851, NDCG@5: 0.89337
SVM - Accuracy: 0.73, NDCG@10: 0.88421, NDCG@5: 0.88001
Gradient Boosting - Accuracy: 0.74, NDCG@10: 0.86196, NDCG@5: 0.85392


### 1. 랜덤 포레스트 (Random Forest)
**장점**:
- 강력한 성능: 다양한 데이터에서 좋은 성능을 보여줍니다.
- 과적합 방지: 여러 결정 트리를 사용하여 과적합을 줄입니다.
- 해석 가능성: 각 트리와 특성의 중요도를 파악할 수 있습니다.

**단점**:
- 느린 예측: 많은 트리를 사용하므로 예측 속도가 느릴 수 있습니다.
- 메모리 사용량: 많은 트리를 저장해야 하므로 메모리 사용량이 높을 수 있습니다.

**NDCG 개선 방법**:
- 더 많은 트리 사용 (n_estimators 증가)
- 최적의 트리 깊이 설정 (max_depth 조정)
- 특성 중요도 분석을 통해 중요한 특성만 사용

### 2. 로지스틱 회귀 (Logistic Regression)
**장점**:
- 간단하고 빠른 모델: 학습과 예측이 빠릅니다.
- 해석 가능성: 모델이 설명하기 쉽습니다.
- 과적합 방지: 규제(regularization)를 사용하여 과적합을 방지합니다.

**단점**:
- 선형성 가정: 특성과 라벨 간의 선형 관계를 가정합니다.
- 비선형 문제에 적합하지 않음: 복잡한 패턴을 잘 포착하지 못할 수 있습니다.

**NDCG 개선 방법**:
- 규제 강도 조정 (C 매개변수 조정)
- 다항 특성(polynomial features) 추가
- 특성 선택(feature selection)을 통해 중요한 특성만 사용

### 3. 서포트 벡터 머신 (SVM)
**장점**:
- 고차원 데이터에서 효과적: 복잡한 결정 경계를 잘 찾습니다.
- 다양한 커널 함수 지원: 비선형 문제도 해결 가능

**단점**:
- 계산 비용: 큰 데이터셋에서 학습 시간이 오래 걸릴 수 있습니다.
- 파라미터 튜닝 필요: 최적의 성능을 위해 커널 및 기타 매개변수 조정 필요

**NDCG 개선 방법**:
- 최적의 커널 함수 선택 (예: RBF, 폴리노미얼)
- 매개변수 조정 (C 및 gamma 값 조정)
- 특성 스케일링 (StandardScaler 사용)

### 4. 그래디언트 부스팅 (Gradient Boosting)
**장점**:
- 높은 성능: 많은 데이터에서 좋은 성능을 보입니다.
- 과적합 조절: 학습률(learning rate)과 트리 개수를 통해 조절 가능
- 특성 중요도 파악 가능

**단점**:
- 느린 학습: 많은 트리를 순차적으로 학습하므로 시간이 오래 걸릴 수 있습니다.
- 파라미터 튜닝 필요: 최적의 성능을 위해 여러 매개변수 조정 필요

**NDCG 개선 방법**:
- 더 많은 트리 사용 (n_estimators 증가)
- 학습률 조정 (learning_rate 조정)
- 트리 깊이 조정 (max_depth 조정)
- 부스팅 유형 변경 (XGBoost 또는 LightGBM 사용)
